In [0]:
from skimage.io import imread
from skimage.measure import block_reduce
from PIL import Image
from os import listdir, path, remove

import numpy as np
from keras.utils import np_utils

In [0]:
def img2array(image_path, ii_size=(64, 64)):
  data = imread(image_path, as_gray=True)
  data = block_reduce(data, block_size=(2, 2))
  return data

In [0]:
def normalize(image_array):
  data = np.asarray(image_array) / 255
  data = np.expand_dims(data, axis = -1)
  return data

In [0]:
def load_data(posix_path):
  X=[]
  for filename in listdir(posix_path):
    if not filename.endswith(".jpg"):
      continue
    p=path.join(posix_path, filename)
    data = img2array(p)
    X.append(data)
  
  return normalize(X)


In [0]:
def gen_labels(length, label):
  return [label for _ in range(length)]

In [0]:
def prepare_image_data():
  x_neg = load_data("data/negatives")
  x_pos = load_data("data/positives")
  y_neg = gen_labels(len(x_neg), 0)
  y_pos = gen_labels(len(x_pos), 1)
  
  X = np.concatenate([x_neg, x_pos])
  Y = np.asarray(y_neg + y_pos)

  X = X.astype(np.float32)
  Y = Y.astype(np.int32)

  inputs=X.shape[1:]

  classes=2
  Y = np_utils.to_categorical(Y, classes).astype(np.float32)

  ixes = np.arange(len(X))
  np.random.shuffle(ixes)
  
  X = X[ixes]
  Y = Y[ixes]
  
  return X, Y, inputs, classes

In [0]:
X, Y, inputs, classes = prepare_image_data()

In [0]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [0]:
def create_smile_cnn(inputs=(32,32,1), classes=2):
  model = Sequential()
  
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=inputs))
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
  
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(classes, activation='softmax'))
  return model

In [0]:
def train_model(train_x, train_y, inputs, classes, epochs, batches):
  model=create_smile_cnn(inputs=(32,32,1), classes=2)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  trs, tt=int(len(train_x)*0.90), int(len(train_x)*0.10)
  train_x, train_y, test_x, test_y=train_x[0:trs], train_y[0:trs], train_x[-tt:], train_y[-tt:]

  model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=epochs, batch_size=batches, verbose=2)
  return model, train_x, train_y, test_x, test_y

In [0]:
train_x, train_y, inputs, classes=prepare_image_data()
model, train_x, train_y, test_x, test_y =train_model(train_x, train_y, inputs, classes, epochs=10, batches=1)